In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle

## Backend

In [ ]:
# backend
# loading single coin models
def coin_model_instance():
    window_size = 60
    model_coin = Sequential([
        LSTM(128, return_sequences=True, input_shape=(window_size, 1)),
        Dropout(0.2),
        LSTM(128, return_sequences=False),
        Dense(units=1)
    ])
    return model_coin


def reload_model(coin):
    window_size = 60
    model_paths = [
        './checkpoints/btc_checkpoint.weights.h5',
        './checkpoints/eth_checkpoint.weights.h5',
        './checkpoints/ltc_checkpoint.weights.h5'
    ]

    model_multicoin = Sequential(
        [LSTM(64, return_sequences=True, input_shape=(window_size, 5)),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dense(units=1)]
    )
    model_multicoin.load_weights('./checkpoints/merge_checkpoint.weights.h5')

    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2

    coin_model = coin_model_instance()
    coin_model.load_weights(model_paths[coin_id])
    # print(model_multicoin.summary())
    # print(coin_model.summary())
    return model_multicoin, coin_model


def reload_scaler(coin):
    coin_paths = [
        './checkpoints/btc_scaler.pkl',
        './checkpoints/ltc_scaler.pkl',
        './checkpoints/eth_scaler.pkl',
    ]

    # multi-coin
    with open('./checkpoints/merge_scaler.pkl', 'rb') as f:
        merge_scaler = pickle.load(f)
        f.close()
    
    # single coin
    coin_id = -1
    if coin == 'BTC':
        coin_id = 0
    elif coin == "LTC":
        coin_id = 1
    elif coin == 'ETH':
        coin_id = 2
    
    with open(coin_paths[coin_id], 'rb') as f_coin:
        coin_scaler = pickle.load(f_coin)
        f_coin.close()

    return merge_scaler, coin_scaler

def fetch_current_data(coin='BTC'):
    btc = yf.Ticker('BTC-USD')
    ltc = yf.Ticker('LTC-USD')
    eth = yf.Ticker('ETH-USD')

    btc_hist = btc.history(period='60D')
    ltc_hist = ltc.history(period='60D')
    eth_hist = eth.history(period='60D')

    coin_list = [btc_hist, ltc_hist, eth_hist]
    for c in coin_list:
        if 'Stock Splits' in c.columns:
            c.drop(columns=['Stock Splits'], inplace=True)
        if 'Dividends' in c.columns:
            c.drop(columns=['Dividends'], inplace=True)

    '''
    For single coin, require only close price data.
    For multi-coin, require all 5 features
    '''

    btc_data = btc_hist
    eth_data = eth_hist
    ltc_data = ltc_hist

    if coin == "BTC":
        return btc_data
    elif coin == "ETH":
        return eth_data
    elif coin == 'LTC':
        return ltc_data


In [ ]:
def forecast(forecast_days=[1, 5, 10, 30], coin='BTC', target_col=3):
    # Fetch initial inputs
    X_merge_init = fetch_current_data(coin)  # shape (60, 5)
    X_coin_init = X_merge_init['Close']      # assuming shape (60, 5) too

    # Reload scalers and models
    merg_scaler, coin_scaler = reload_scaler(coin)
    merge_model, coin_model = reload_model(coin)

    # Constants
    total_features = X_merge_init.shape[-1]

    # Convert to NumPy arrays
    X_input_multicoin = X_merge_init.to_numpy()
    X_input_coin = X_merge_init['Close'].to_numpy()

    # Ensure shapes are correct
    X_input_multicoin = X_input_multicoin.reshape(1, 60, total_features)
    X_input_coin = X_input_coin.reshape(1, 60, 1)

    # Store predictions
    forecast_result_multicoin = {}
    forecast_result_single = {}
    predicted_closes_multicoin = []
    predicted_closes_single = []

    for _ in range(1, max(forecast_days) + 1):
        # Predict scaled close values
        pred_scaled_multicoin = merge_model.predict(X_input_multicoin, verbose=0)
        pred_scaled_coin = coin_model.predict(X_input_coin, verbose=0)

        # Inverse transform preparation
        expanded_multicoin = np.zeros((1, total_features))
        expanded_multicoin[:, target_col] = pred_scaled_multicoin.flatten()

        expanded_coin = np.zeros((1, total_features))
        expanded_coin[:, target_col] = pred_scaled_coin.flatten()

        # Inverse transform
        pred_full_multicoin = merg_scaler.inverse_transform(expanded_multicoin)
        pred_full_coin = coin_scaler.inverse_transform(expanded_coin)

        pred_close_multicoin = pred_full_multicoin[:, target_col][0]
        pred_close_coin = pred_full_coin[:, target_col][0]

        predicted_closes_multicoin.append(pred_close_multicoin)
        predicted_closes_single.append(pred_close_coin)

        # Recursive input update
        next_input_multi = X_input_multicoin[:, 1:, :].copy()
        new_step_multi = X_input_multicoin[:, -1, :].copy()
        new_step_multi[0, target_col] = pred_scaled_multicoin.flatten()[0]
        X_input_multicoin = np.concatenate([next_input_multi, new_step_multi.reshape(1, 1, total_features)], axis=1)

        next_input_coin = X_input_coin[:, 1:, :].copy()
        new_step_coin = X_input_coin[:, -1, :].copy()
        new_step_coin[:, 0] = pred_scaled_coin.flatten()
        X_input_coin = np.concatenate([next_input_coin, new_step_coin.reshape(1, 1, 1)], axis=1)


    # Extract forecasts
    for d in forecast_days:
        forecast_result_multicoin[f"{d}d"] = predicted_closes_multicoin[d - 1]
        forecast_result_single[f"{d}d"] = predicted_closes_single[d - 1]
    
    current_close_price = X_input_multicoin[:, -1, 3].item()

    # Return predictions and the latest observed timestep
    return forecast_result_multicoin, forecast_result_single, current_close_price



In [6]:
def generate_recommendation(predictions, current_price):
    returns = {k: ((v - current_price) / current_price) * 100 for k, v in predictions.items()}
    sorted_days = sorted(returns, key=lambda x: int(x[:-1])) 

    all_negative = all(r < 0 for r in returns.values())
    all_positive = all(r > 2 for r in returns.values())
    immediate_gain = returns["1d"] > 1
    rebound_after_drop = returns["1d"] < -1 and returns["5d"] > 3

    if all_negative:
        return "Sell now"
    elif rebound_after_drop:
        return "Buy more and sell on day 5"
    elif immediate_gain and returns["30d"] > 8:
        return "Buy more and hold indefinitely"
    elif returns["1d"] > 0 and returns["5d"] < 0:
        return "Hold now and sell on day 1"
    elif returns["1d"] < 1 and returns["30d"] > 5:
        return "Hold now and buy more"
    elif max(returns.values()) < 2:
        return "Hold indefinitely"
    else:
        best_day = max(returns, key=returns.get)
        return f"Buy more and sell on day {best_day}"

def get_optimal_prediction(coin):
    
    # getting forcasts
    forecast_multicoin, forcast_single, current_values = forecast(coin=coin)
    # print(current_values)
    current_price = current_values
    rec_multicoin = generate_recommendation(forecast_multicoin, current_price)
    rec_single = generate_recommendation(forcast_single, current_price)

    text = f"""\n
    Forcast & Recommendations for {coin}:
    Based of the Mutlicoin model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close price     {forecast_multicoin['1d']:.4f}     {forecast_multicoin['5d']:.4f}     {forecast_multicoin['10d']:.4f}    {forecast_multicoin['30d']:.4f}

    Recommendation: {rec_multicoin}

    Based of the {coin} model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close price     {forcast_single['1d']:.4f}     {forcast_single['5d']:.4f}     {forcast_single['10d']:.4f}    {forcast_single['30d']:.4f}

    Recommendation: {rec_single}
    """
    print(text)


## Frontend

In [ ]:
# frontend
print("Welcome to Crypto Guide")
print("""
In this guide, users can select and forsee the forcast of 3 different coins by inbuilt models. 
Coin selection can be made from the following: Bitcoin, Ethereum, Litecoin.
The program also gives out recommendations based on forecasts of the individual coin.

Disclaimer on Prediction Discrepancies:
Predictions made by the single-coin model and the multi-coin model may differ significantly due to differences in the data used and the assumptions each model captures.
Single coin model has been trained exclusively on the histrocial data of a single cryptocurrency. Henceforth, it is more tailored towards capturing that coin's trends, seasonality and behavior.
Mutlicoin has been trained on a merged dataset consisting historical price data of BTC, ETH and LTC. It learns the general market trend.
""")
while True:
    try:
        user_coin = str(input('Enter coin [BTC: Bitcoin, ETH: Ethereum, LTC: Litcoin]'))
        user_coin = user_coin.strip()
        # user_coin = "BTC"
        if user_coin not in ["BTC", "ETH", "LTC"]:
            print("Coin not found!")
        else:
            get_optimal_prediction(user_coin)

            res = int(input('Do you want to any other coin? (Yes: 1, No: 0): '))
            # res = res.strip()
            # print("res: ", res)
            if res == 1:
                continue
            else:
                break
    except Exception as e:
        raise e

print("Thank you")

Welcome to Crypto Guide

In this guide, users can select and forsee the forcast of 3 different coins by inbuilt models. 
Coin selection can be made from the following: Bitcoin, Ethereum, Litecoin.
The program also gives out recommendations based on forecasts of the individual coin.

Disclaimer on Prediction Discrepancies:
Predictions made by the single-coin model and the multi-coin model may differ significantly due to differences in the data used and the assumptions each model captures.
Single coin model has been trained exclusively on the histrocial data of a single cryptocurrency. Henceforth, it is more tailored towards capturing that coin's trends, seasonality and behavior.
Mutlicoin has been trained on a merged dataset consisting historical price data of BTC, ETH and LTC. It learns the general market trend.



    Forcast & Recommendations for BTC:
    Based of the Mutlicoin model: 
    Forcast:
                    1 Day           5 Day           10 Day          30 Day
    Close p